# Scripts :

## Requête pour interroger l'API et insérer les données de 3 stations :

---

In [ ]:
def loop():

    import sqlite3
    import requests
    import json
    from datetime import datetime
    
    station = [410, 432, 113]

    foil = []
    
    for e in station :
    
        requete = f"http://api.pioupiou.fr/v1/live-with-meta/{e}"

        response = requests.get(requete)
        
        response_dict = response.json()
    
        foil.append(response.json())
    
    db = sqlite3.connect('winds.db')

    cursor = db.cursor()

    for data in foil:
    
        data = data['data']
    
        cursor.execute("""INSERT INTO measurement (id,
                                                   latitude,
                                                   longitude,
                                                   state,
                                                   date,
                                                   wind_heading,
                                                   wind_speed_avg,
                                                   wind_speed_max,
                                                   wind_speed_min)
                          VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)""",
                         (data['id'],
                          data['location']['latitude'],
                          data['location']['longitude'],
                          data['status']['state'],
                          data['measurements']['date'],
                          data['measurements']['wind_heading'],
                          data['measurements']['wind_speed_avg'],
                          data['measurements']['wind_speed_max'],
                          data['measurements']['wind_speed_min'])
                       )
                                        
        db.commit()
        
        print("Insertion effectuée", datetime.now())
        
    cursor.close()
    db.close()

---

## Requête pour créer une boucle d'insertion automatique dans la BDD

In [ ]:
def schedule_loop():

    from schedule import schedule
    import time
    
    #if:
    
    #else :
    loop()
    schedule.every(1).minutes.do(loop)
    schedule.every(140).seconds.do(create_copy_db)
    schedule.every(140).seconds.do(create_readable_dbcopy)
        
    while True:
        schedule.run_pending()
        time.sleep(10)

In [ ]:
schedule_loop()

---

## Requêtes pour sauvegarder la BDD

In [ ]:
def create_readable_dbcopy():
    
    #create a .sql file as a backup in Backup folder
    import sqlite3
    import io
    from datetime import datetime

    conn = sqlite3.connect('winds.db')
    with io.open('Backup/winds_dump.sql', 'w') as f:
        for measurement in conn.iterdump():
            f.write('%s\n' % measurement)
            
    print('Backup performed successfully.', datetime.now())
    
    conn.close()

In [ ]:
create_readable_dbcopy()

In [ ]:
def create_copy_db():

    #create a copy of the database as a backup in Backup folder
    import shutil
    import os
    from datetime import datetime
 
    try:
        path='C:/Users/joris/Documents/Simplon/Dev_data/Vents/Foil'
    
        shutil.copyfile('winds.db','C:/Users/joris/Documents/Simplon/Dev_data/Vents/Foil/Backup/winds.db')
 
        print("Success copying winds.db", datetime.now())
    
    except:
        print("error while copying winds.db")

In [ ]:
create_copy_db()

---

## Requêtes interrogation bdd pour supprimer 3 lignes

In [ ]:
import sqlite3

def check_row_number():
    with sqlite3.connect("winds.db") as db:
        cursor = db.cursor()
        cursor.execute('''SELECT COUNT(rowid) from measurement ''')
        count = cursor.fetchall()
        return (count[0][0])

In [ ]:
check_row_number()

In [ ]:
def del_rows():

    from datetime import datetime
    
    with sqlite3.connect("winds.db") as db:
        cursor = db.cursor()
        cursor.execute('''DELETE
                          FROM measurement
                          WHERE rowid IN (Select rowid from measurement limit 3);
                        ''')
        print("delete success", datetime.now())

In [ ]:
del_rows()

---

## Création application

In [ ]:
import schedule
import time

def application():

    if check_row_number() > 12 :
        del_rows()
        print ("delete")
        print (check_row_number())
            
    else :
        
        schedule.every(1).minutes.do(loop)
        schedule.every(130).seconds.do(create_copy_db)
        schedule.every(130).seconds.do(create_readable_dbcopy)
        
        while True:
            schedule.run_pending()
            time.sleep(1)

In [ ]:
application()

In [ ]:
def app():

    from datetime import datetime
    
    if check_row_number() > 9 :
        del_rows()
        print ("Delete", check_row_number())
    
    else :
        print ("Nothing to delete", datetime.now())

In [ ]:
import schedule
import time

app()
loop()       
schedule.every(50).seconds.do(app)
schedule.every(1).minutes.do(loop)
schedule.every(130).seconds.do(create_copy_db)
schedule.every(130).seconds.do(create_readable_dbcopy)

while True:
    schedule.run_pending()
    time.sleep(1)

In [ ]:
import utils

utils.check_row_number()

In [1]:
import utils

utils.foil()

Delete succeed : 9  rows in Database 2021-03-12 14:49:15.587099
Insertion effectuée 2021-03-12 14:49:15.752947
Insertion effectuée 2021-03-12 14:49:15.758835
Insertion effectuée 2021-03-12 14:49:15.763799
Delete succeed : 9  rows in Database 2021-03-12 14:50:06.282634
Insertion effectuée 2021-03-12 14:50:16.610751
Insertion effectuée 2021-03-12 14:50:16.615737
Insertion effectuée 2021-03-12 14:50:16.620551
Delete succeed : 9  rows in Database 2021-03-12 14:50:57.057954
Insertion effectuée 2021-03-12 14:51:17.448197
Insertion effectuée 2021-03-12 14:51:17.454167
Insertion effectuée 2021-03-12 14:51:17.459211
Success copying winds.db 2021-03-12 14:51:26.537591
Backup performed successfully. 2021-03-12 14:51:26.539561
Delete succeed : 9  rows in Database 2021-03-12 14:51:47.773074
Insertion effectuée 2021-03-12 14:52:18.222225
Insertion effectuée 2021-03-12 14:52:18.227826
Insertion effectuée 2021-03-12 14:52:18.232812
Delete succeed : 9  rows in Database 2021-03-12 14:52:38.464200
Insert

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\joris\documents\simplon\dev_data\vents\venvp39\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-07e0bbcc4cd0>", line 3, in <module>
    utils.foil()
  File "C:\Users\joris\Documents\Simplon\Dev_data\Vents\Foil\utils.py", line 151, in foil
    loop()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\joris\documents\simplon\dev_data\vents\venvp39\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\joris\documents\simplon\dev_data\vents\venvp39\lib\site-packages\IPython\core\ultrat

TypeError: object of type 'NoneType' has no len()